# NL2SQL leveraging the Azure Accelerator

In [ ]:
import os
import pandas as pd
import pyodbc
from langchain_openai import AzureChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent, SQLDatabaseToolkit
# from langchain.agents import create_sql_agent 
from langchain.agents import initialize_agent
from langchain.agents import Tool
from langchain.agents import create_sql_agent  # No longer in agent_toolkits
from langchain.sql_database import SQLDatabase  # Toolkit is replaced with SQLDatabase
from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager
from common.prompts import MSSQL_AGENT_PREFIX
from IPython.display import Markdown, HTML, display  
from dotenv import load_dotenv

load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))

printmd("##### Depedencies loaded...")

In [ ]:
# Set environment variables
app_env = os.environ["AZURE_OPENAI_API_VERSION"]

print(f"app_env: {app_env}")

In [ ]:
# Configure Logging
import logging

# Define a custom log format
log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'

# logging.basicConfig(level=logging.DEBUG)

logging.basicConfig(level=logging.DEBUG, format=log_format)
logger = logging.getLogger('langchain')

printmd("##### Logger created...")

In [ ]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
import os

# Configuration for the database connection
db_config = {
    'drivername': 'mssql+pyodbc',
    'username': os.environ["SQL_SERVER_USERNAME"] + '@' + os.environ["SQL_SERVER_NAME"],
    'password': os.environ["SQL_SERVER_PASSWORD"],
    'host': os.environ["SQL_SERVER_NAME"],
    'port': 1433,
    'database': os.environ["SQL_SERVER_DATABASE"],
    'query': {'driver': 'ODBC Driver 17 for SQL Server'},
}

# Create a URL object for connecting to the database
db_url = URL.create(**db_config)

# Connect to the Azure SQL Database using the URL string
engine = create_engine(db_url)

# Test the connection using the SQLAlchemy 2.0 execution style
with engine.connect() as conn:
    try:
        # Use the text() construct for safer SQL execution
        result = conn.execute(text("SELECT @@VERSION"))
        version = result.fetchone()
        print("Connection successful!")
        print(version)
    except Exception as e:
        print(e)

In [ ]:
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=2000, api_version=os.environ["AZURE_OPENAI_API_VERSION"])
# llm = AzureChatOpenAI(deployment_name=os.environ["AZURE_OPENAI_API_VERSION"], temperature=0.5, max_tokens=2000)

print(llm)

In [ ]:
# Let's create the db object
db = SQLDatabase.from_uri(db_url)

# print(db.get_table_names())
print("SqlAgent Created")
print("Found following tables:")
print(db.get_usable_table_names())

In [ ]:
# Natural Language question (query)
QUESTION = """
How may patients were hospitalized during July 2020 in Texas. 
And nationwide as the total of all states? 
Use the hospitalizedIncrease column
"""
printmd(f"{QUESTION}")

In [ ]:
# Natural Language question (query)
QUESTION = """
How many products are available in the inventory?
"""
printmd(f"{QUESTION}")

In [ ]:
# Natural Language question (query)
QUESTION = """
List the all of the rock products
"""
printmd(f"{QUESTION}")

In [ ]:
# Natural Language question (query)
QUESTION = """
What products include Bruce Springsteen?
"""
printmd(f"{QUESTION}")

In [ ]:
# Create Callback Manager to capture SQL queries
from langchain.callbacks.base import BaseCallbackHandler

class SQLCallbackHandler(BaseCallbackHandler):
    def on_tool_end(self, output, color="green", **kwargs):
        """Run when tool ends running."""
        print(f"SQL Query: {output}")

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

try:
    agent_executor = create_sql_agent(
        prefix=MSSQL_AGENT_PREFIX,
        llm=llm,
        toolkit=toolkit,
        top_k=30,
        agent_type="openai-tools",
        verbose=True,
        callback_manager=CallbackManager([SQLCallbackHandler()]),
        agent_executor_kwargs={"return_intermediate_steps": True},
    )
except Exception as e:
    print(f"An error occurred while creating the agent_executor: {str(e)}")
    agent_executor = None  # Set agent_executor to None or handle it as needed

# Continue with the rest of your code, checking if agent_executor was created successfully
if agent_executor:
    printmd(f"##### Agent Executor Created")

In [ ]:
# As we know by now, Agents use expert/tools. Let's see which are the tools for this SQL Agent
agent_executor.tools

In [ ]:
show_details = False

try:
    response = agent_executor.invoke(QUESTION) 
except Exception as e:
    response = str(e)

# printmd(f"**Response:** {response}")

# for step in response['intermediate_steps']:
#     action = step.get('action', '')  # Use .get to handle missing keys
#     observation = step.get('observation', '')
#     print(f"Action: {action}\nObservation: {observation}\n")

if show_details: 

    for step in response['intermediate_steps']:
        if isinstance(step, dict):
            action = step.get('action', '')
            observation = step.get('observation', '')
        elif isinstance(step, tuple):
            action, observation = step 
        else:
            print(f"Unexpected step type: {type(step)}")
            continue  # Skip this step if it's neither dict nor tuple
        print(f"Action: {action}\nObservation: {observation}\n")
else:
    print("No details to show")

# print(response['intermediate_steps'])

In [ ]:
print (response["output"])